In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import xgboost as xgb
import lightgbm as lgb
from datetime import datetime
from catboost import Pool, CatBoostClassifier
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_squared_error
from math import sqrt
import math
from imblearn.over_sampling import SMOTE

In [2]:
train = pd.read_csv("train.csv")
train.set_index('EmployeeNo', inplace=True)
test = pd.read_csv("test.csv")
test.set_index('EmployeeNo', inplace=True)

In [3]:
train.head()

,Division,Qualification,Gender,Channel_of_Recruitment,Trainings_Attended,Year_of_birth,Last_performance_score,Year_of_recruitment,Targets_met,Previous_Award,Training_score_average,State_Of_Origin,Foreign_schooled,Marital_Status,Past_Disciplinary_Action,Previous_IntraDepartmental_Movement,No_of_previous_employers,Promoted_or_Not
EmployeeNo,,,,,,,,,,,,,,,,,,
YAK/S/00001,Commercial Sales and Marketing,"MSc, MBA and PhD",Female,Direct Internal process,2,1986,12.5,2011,1,0,41,ANAMBRA,No,Married,No,No,0,0
YAK/S/00002,Customer Support and Field Operations,First Degree or HND,Male,Agency and others,2,1991,12.5,2015,0,0,52,ANAMBRA,Yes,Married,No,No,0,0
YAK/S/00003,Commercial Sales and Marketing,First Degree or HND,Male,Direct Internal process,2,1987,7.5,2012,0,0,42,KATSINA,Yes,Married,No,No,0,0
YAK/S/00004,Commercial Sales and Marketing,First Degree or HND,Male,Agency and others,3,1982,2.5,2009,0,0,42,NIGER,Yes,Single,No,No,1,0
YAK/S/00006,Information and Strategy,First Degree or HND,Male,Direct Internal process,3,1990,7.5,2012,0,0,77,AKWA IBOM,Yes,Married,No,No,1,0


In [4]:
train['is_train'] = 0
test['is_train'] = 1
train_y = train['Promoted_or_Not']

In [5]:
# Create dummy variables for every unique category
le = LabelEncoder()
train["Division"] = le.fit_transform(train["Division"])
train["Gender"] = le.fit_transform(train["Gender"])
train["Channel_of_Recruitment"] = le.fit_transform(train["Channel_of_Recruitment"])
train["Foreign_schooled"] = le.fit_transform(train["Foreign_schooled"])
train["Marital_Status"] = le.fit_transform(train["Marital_Status"])
train["Past_Disciplinary_Action"] = le.fit_transform(train["Past_Disciplinary_Action"])
train["State_Of_Origin"] = le.fit_transform(train["State_Of_Origin"])
train["Previous_IntraDepartmental_Movement"] = le.fit_transform(train["Previous_IntraDepartmental_Movement"])
train['Qualification'] = train['Qualification'].fillna('First Degree or HND')
train['Qualification'] = le.fit_transform(train['Qualification'])
train['No_of_previous_employers'] = train['No_of_previous_employers'].replace('More than 5', 6)
train['No_of_previous_employers'] = pd.to_numeric(train['No_of_previous_employers'])


le = LabelEncoder()
test["Division"] = le.fit_transform(test["Division"])
test["Gender"] = le.fit_transform(test["Gender"])
test["Channel_of_Recruitment"] = le.fit_transform(test["Channel_of_Recruitment"])
test["Foreign_schooled"] = le.fit_transform(test["Foreign_schooled"])
test["Marital_Status"] = le.fit_transform(test["Marital_Status"])
test["Past_Disciplinary_Action"] = le.fit_transform(test["Past_Disciplinary_Action"])
test["State_Of_Origin"] = le.fit_transform(test["State_Of_Origin"])
test["Previous_IntraDepartmental_Movement"] = le.fit_transform(test["Previous_IntraDepartmental_Movement"])
test['Qualification'] = test['Qualification'].fillna('First Degree or HND')
test['Qualification'] = le.fit_transform(test['Qualification'])
test['No_of_previous_employers'] = test['No_of_previous_employers'].replace('More than 5', 6)
test['No_of_previous_employers'] = pd.to_numeric(test['No_of_previous_employers'])

In [6]:
train = train.drop('Promoted_or_Not',axis=1)

df = pd.concat([train, test], axis=0)

print(train.shape)

(38312, 18)


In [7]:
# df = df[df['is_train'] == 0].drop(['is_train',],axis=1)
df.head()

,Division,Qualification,Gender,Channel_of_Recruitment,Trainings_Attended,Year_of_birth,Last_performance_score,Year_of_recruitment,Targets_met,Previous_Award,Training_score_average,State_Of_Origin,Foreign_schooled,Marital_Status,Past_Disciplinary_Action,Previous_IntraDepartmental_Movement,No_of_previous_employers,is_train
EmployeeNo,,,,,,,,,,,,,,,,,,
YAK/S/00001,1,1,0,1,2,1986,12.5,2011,1,0,41,3,0,0,0,0,0,0
YAK/S/00002,2,0,1,0,2,1991,12.5,2015,0,0,52,3,1,0,0,0,0,0
YAK/S/00003,1,0,1,1,2,1987,7.5,2012,0,0,42,20,1,0,0,0,0,0
YAK/S/00004,1,0,1,0,3,1982,2.5,2009,0,0,42,26,1,2,0,0,1,0
YAK/S/00006,4,0,1,1,3,1990,7.5,2012,0,0,77,2,1,0,0,0,1,0


In [8]:
sm = SMOTE(random_state=1)
train, train_y = sm.fit_sample(train, train_y)
print(train.shape)

(70142, 18)


In [9]:
train_y.shape

(70142,)

In [10]:
numericals = df[['Trainings_Attended', 'Last_performance_score', 'Targets_met','Previous_Award','Training_score_average', 'No_of_previous_employers']]
df[['Trainings_Attended', 'Last_performance_score', 'Targets_met','Previous_Award','Training_score_average', 'No_of_previous_employers']] = preprocessing.MinMaxScaler().fit_transform(numericals)

In [11]:
df.head()

,Division,Qualification,Gender,Channel_of_Recruitment,Trainings_Attended,Year_of_birth,Last_performance_score,Year_of_recruitment,Targets_met,Previous_Award,Training_score_average,State_Of_Origin,Foreign_schooled,Marital_Status,Past_Disciplinary_Action,Previous_IntraDepartmental_Movement,No_of_previous_employers,is_train
EmployeeNo,,,,,,,,,,,,,,,,,,
YAK/S/00001,1,1,0,1,0.000000,1986,1.0,2011,1.0,0.0,0.166667,3,0,0,0,0,0.000000,0
YAK/S/00002,2,0,1,0,0.000000,1991,1.0,2015,0.0,0.0,0.350000,3,1,0,0,0,0.000000,0
YAK/S/00003,1,0,1,1,0.000000,1987,0.6,2012,0.0,0.0,0.183333,20,1,0,0,0,0.000000,0
YAK/S/00004,1,0,1,0,0.111111,1982,0.2,2009,0.0,0.0,0.183333,26,1,2,0,0,0.166667,0
YAK/S/00006,4,0,1,1,0.111111,1990,0.6,2012,0.0,0.0,0.766667,2,1,0,0,0,0.166667,0


In [12]:
train_x = df

In [13]:
test_x = train_x[train_x['is_train'] == 1]
# Test before Train !!
train_x = train_x[train_x['is_train'] == 0]

train_x = train_x.drop(['is_train',],axis=1)
test_x = test_x.drop(['is_train',],axis=1)

In [15]:
rmse = lambda x, y: math.sqrt(mean_squared_error(x, y))

train_x.shape

(38312, 17)

In [ ]:
train_pool = Pool(train_x, train_y) 
test_pool = Pool(test_x)
cbm0 = CatBoostClassifier(rsm=0.2, depth=13, learning_rate=0.2)

In [ ]:
# A parameter grid for XGBoost
params = {'min_child_weight':[4,5], 
          'nthread':[4],
          'gamma':[i/10.0 for i in range(3,6)],  
          'learning_rate': [.03, 0.05, .07, 0.1],
          'subsample':[i/10.0 for i in range(6,11)],
          'min_child_weight': [4],
          'silent': [1],
          'colsample_bytree':[i/10.0 for i in range(6,11)], 
          'max_depth': [2,3,4],
          'n_estimators': [1000]}

#XGBClassifier
xgb0 = xgb.XGBClassifier(nthread=-1,objective='binary:logistic',missing=None,seed=8)
xgb0 = RandomizedSearchCV(xgb0, params, n_jobs=1,n_iter=10)

gridParams = {
    'learning_rate': [0.005,0.2],
    'n_estimators': [40],
    'num_leaves': [16,20,40,55,65],
    'boosting_type' : ['gbdt'],
    'objective' : ['binary'],
    'random_state' : [501], # Updated from 'seed'
    'colsample_bytree' : [0.65, 0.66],
    'subsample' : [0.7,0.75],
    'reg_alpha' : [1,1.2],
    'reg_lambda' : [1,1.2,1.4],
    }
#LGBMRegressor
gbm0 = lgb.LGBMClassifier()
gbm0 = RandomizedSearchCV(gbm0, gridParams,cv=4,n_jobs=2)

In [ ]:
from sklearn.ensemble import VotingClassifier

elect = VotingClassifier(estimators=[
        ('cbm', cbm0), ('xgb', xgb0), ('gbm', gbm0)], voting='soft')
elect = elect.fit(train_x, train_y)

In [ ]:
elect_predictions = elect.predict(test_x)

In [ ]:
submit = pd.DataFrame(
   {
       'EmployeeNo':test.index,
       'Promoted_or_Not':elect_predictions
   }
)


submit.to_csv('vote.csv',index=False)

In [ ]:
predict_y1 = gbm0.predict(test_x)
predict_y2 = xgb0.predict(test_x)
predict_y3 = cbm0.predict(test_pool)

In [ ]:
predict_y = (predict_y1+predict_y2+predict_y3)

In [ ]:
predict_y1

In [ ]:
predict_y = [int(round(value)) for value in predict_y]

In [ ]:
print("Fitting CatBoostRegressor model...")
cbm0.fit(train_pool)
print("Finished fitting CatBoostRegressor model")

print("Fitting XGBRegressor model...")
xgb0.fit(train_x, train_y)
print("Finished fitting XGBRegressor model")

print("Fitting LGBMRegressor model...")
gbm0.fit(train_x, train_y, eval_metric='rmse')
print("Finished fitting LGBMRegressor model")

In [ ]:
final = pd.read_csv("submit.csv")
final.head()

In [ ]:
# A parameter grid for XGBoost
params = {'min_child_weight':[4,5], 
          'nthread':[4],
          'gamma':[i/10.0 for i in range(3,6)],  
          'learning_rate': [.03, 0.05, .07, 0.1],
          'subsample':[i/10.0 for i in range(6,11)],
          'min_child_weight': [4],
          'silent': [1],
          'colsample_bytree':[i/10.0 for i in range(6,11)], 
          'max_depth': [2,3,4],
          'n_estimators': [1000]}

my_XGB_model = xgb.XGBRegressor(nthread=-1,objective='binary:logistic',missing=None,seed=8)
grid = RandomizedSearchCV(my_XGB_model, params, n_jobs=1,n_iter=10)
grid.fit(train_x, train_y)

In [ ]:
XGB_predictions1 = grid.predict(test_x)
XGB_predictions = [int(round(value)) for value in XGB_predictions1]
submit = pd.DataFrame(
   {
       'EmployeeNo':test.index,
       'Promoted_or_Not':XGB_predictions
   }
)


submit.to_csv('submitxgb.csv',index=False)